In [16]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [17]:
# drive access
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

!pip install transformers
from transformers import RobertaTokenizer, TFAutoModel, AdamWeightDecay

import tensorflow as tf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd

# Load the CSV file
file_path = '/content/drive/MyDrive/Dataset/thesis_dataset.csv'
dataset = pd.read_csv(file_path)
df = dataset.copy()

def modify_df(dataset):

    df['Labels'] = df['Labels'].apply(lambda x: 0 if x == 'no' else x)
    df['Labels'] = df['Labels'].apply(lambda x: 1 if x == 'postpartum' else x)
    df['Labels'] = df['Labels'].apply(lambda x: 2 if x == 'major depressive' else x)
    df['Labels'] = df['Labels'].apply(lambda x: 3 if x == 'bipolar' else x)
    df['Labels'] = df['Labels'].apply(lambda x: 4 if x == 'psychotic' else x)
    df['Labels'] = df['Labels'].apply(lambda x: 5 if x == 'atypical' else x)

    return df

In [36]:
from nltk.corpus import stopwords
def clean_text(text):
    if isinstance(text, str):  # Check if text is a string
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
        return text
    else:
        return ''

df['Tweets'] = df['Tweets'].apply(clean_text)
df.drop(df[df['Tweets'] == ''].index, inplace=True)
df.dropna(inplace=True)

In [37]:
df_postpartum = df[df['Labels'] == 'postpartum']
df_major = df[df['Labels'] == 'major depressive']
df_psychotic = df[df['Labels'] == 'psychotic']
df_bipolar = df[df['Labels'] == 'bipolar']
df_no_dep = df[df['Labels'] == 'no']
df_atypical = df[df['Labels'] == 'atypical']

In [38]:
df_postpartum_downsampled = df_postpartum.sample(1000)
df_postpartum_downsampled.shape

(1000, 2)

In [39]:
df_major_downsampled = df_major.sample(1000)
df_major_downsampled.shape

(1000, 2)

In [40]:
df_psychotic_downsampled = df_psychotic.sample(1000)
df_psychotic_downsampled.shape

(1000, 2)

In [41]:
df_bipolar_downsampled = df_bipolar.sample(1000)
df_bipolar_downsampled.shape

(1000, 2)

In [42]:
df_no_dep_downsampled = df_no_dep.sample(1000)
df_no_dep_downsampled.shape

(1000, 2)

In [43]:
df_atypical_downsampled = df_atypical.sample(1000)
df_atypical_downsampled.shape

(1000, 2)

In [44]:
df_balanced = pd.concat([df_postpartum_downsampled, df_major_downsampled, df_bipolar_downsampled, df_psychotic_downsampled, df_no_dep_downsampled, df_atypical_downsampled])
df_balanced.shape

(6000, 2)

In [45]:
df_balanced['Labels'].value_counts()

Labels
postpartum          1000
major depressive    1000
bipolar             1000
psychotic           1000
no                  1000
atypical            1000
Name: count, dtype: int64

In [68]:
def df_info(df_balanced,name):
    no = df_balanced[df_balanced.Labels == 0].shape[0]
    postpartum = df_balanced[df_balanced.Labels == 1].shape[0]
    major_depressive = df_balanced[df_balanced.Labels == 2].shape[0]
    bipolar = df_balanced[df_balanced.Labels == 3].shape[0]
    psychotic = df_balanced[df_balanced.Labels == 4].shape[0]
    atypical = df_balanced[df_balanced.Labels == 5].shape[0]

    print(f"\n{name} -----> shape : {df_balanced.shape}")
    print(f"{name} -----> no: {no}, postpartum: {postpartum}, major_depressive: {major_depressive}, bipolar: {bipolar}, psychotic: {psychotic}, atypical: {atypical}\n")

    return None

In [69]:
train, valid_test = train_test_split(df_balanced, test_size = 0.1, shuffle=True)
valid, test = train_test_split(valid_test, test_size = 0.5, shuffle=True)

df_info(train, "train")
df_info(valid, "valid")
df_info(test, "test")

x_train = train[["Tweets"]]
y_train = train[["Labels"]]

x_valid = valid[["Tweets"]]
y_valid = valid[["Labels"]]

x_test = test[["Tweets"]]
y_test = test[["Labels"]]



train -----> shape : (5400, 2)
train -----> no: 0, postpartum: 0, major_depressive: 0, bipolar: 0, psychotic: 0, atypical: 0


valid -----> shape : (300, 2)
valid -----> no: 0, postpartum: 0, major_depressive: 0, bipolar: 0, psychotic: 0, atypical: 0


test -----> shape : (300, 2)
test -----> no: 0, postpartum: 0, major_depressive: 0, bipolar: 0, psychotic: 0, atypical: 0



In [70]:
def preprocess(df_balanced, text_col_name):
    x_train = preprocess(x_train.copy(), "Tweets")
    x_valid = preprocess(x_valid.copy(), "Tweets")
    x_test = preprocess(x_test.copy(), "Tweets")
print(x_train.head())

                                                  Tweets
7865   wondering bought wilson tix thru libs barrell ...
383    im sad tik tok got suspended getting views fee...
548    im weird 槀槀 depressed whole week discovered po...
12306  tears come easily days sometimes dont even kno...
3654   today shit news got called ugly got told thera...


In [71]:
SEQ_LEN = 80

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [72]:
def tokenize(df_balanced):
    input_ids =  []
    attention_masks =  []

    for i, text in enumerate(df["Tweets"]):
        tokens = tokenizer.encode_plus(text, max_length=SEQ_LEN,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')

        input_ids.append(np.asarray(tokens["input_ids"]).reshape(SEQ_LEN,))
        attention_masks.append(np.asarray(tokens["attention_mask"]).reshape(SEQ_LEN,))

    return (np.asarray(input_ids), np.asarray(attention_masks))

train_input_ids, train_attention_masks = tokenize(x_train)
valid_input_ids, valid_attention_masks = tokenize(x_valid)
test_input_ids, test_attention_masks = tokenize(x_test)

print(train_input_ids[0])

[    0 12891 34876  6943 46405   190  4883  6612   891   688   536   460
    26  4356   670  4356  1613   213 34876   300  8597   464   809  1022
  1508  1022  1928   101  3148  1086  1050    66  4349   240    86     2
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1]


In [62]:
def one_encode_labels(df_balanced):
    sentiment_values = list(set(df_balanced["Labels"].values))
    labels = []
    for index, row in df_balanced.iterrows():
        label = np.zeros((len(sentiment_values)))
        label[row["Labels"]] = 1
        labels.append(label)
    labels = np.asarray(labels)
    return labels

In [63]:
y_train = one_encode_labels(y_train)
y_valid = one_encode_labels(y_valid)

print(y_train[0])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [64]:
roberta = TFAutoModel.from_pretrained("roberta-base")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [65]:

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = roberta(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.LSTM(128)(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(768)(X)
X = tf.keras.layers.Activation("relu")(X)
X = tf.keras.layers.Dense(768)(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 80)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 80)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_roberta_model (TFRobert  TFBaseModelOutputWithPooli   1246456   ['input_ids[0][0]',           
 aModel)                     ngAndCrossAttentions(last_   32         'attention_mask[0][0]']      
                             hidden_state=(None, 80, 76                                       

In [66]:
optimizer = AdamWeightDecay(2e-03, beta_1=0.8, beta_2=0.9, weight_decay_rate=0.0001)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [67]:
history = model.fit((train_input_ids, train_attention_masks),y_train, validation_data=((valid_input_ids,valid_attention_masks), y_valid), epochs=10, batch_size=32)

ValueError: Data cardinality is ambiguous:
  x sizes: 14983, 14983
  y sizes: 5400
Make sure all arrays contain the same number of samples.